# __Long Short Term Memory Decoder__

### __Deep Learning__

#### __Project: Image Captioning with Visual Attention__

In [1]:
import os
os.chdir(os.environ["PYTHONPATH"])

import torch
import matplotlib.pyplot as plt
import seaborn as sns

import scripts.data_loading as dl
import scripts.data_preprocessing as dp
from scripts import model

%matplotlib inline
%load_ext autoreload
%autoreload 2


plt.rcParams["figure.figsize"] = (10, 10)
plt.rcParams["image.cmap"] = "plasma"

In [2]:
coco_train = dl.CocoCaptions(
    dl.TRAINING_DATASET_PATHS[dl.DatasetType.TRAIN],
    dp.VGGNET_PREPROCESSING_PIPELINE,
    dp.TextPipeline(),
)

loading annotations into memory...
Done (t=2.00s)
creating index...
index created!


In [3]:
coco_loader = dl.CocoLoader(coco_train, batch_size=2, num_workers=1)
it = iter(coco_loader)
image_batch, caption_batch = next(it)

In [4]:
encoder = model.VGG19Encoder()
feature_maps, feature_mean = encoder.forward(image_batch)

In [5]:
decoder = model.LSTMDecoder(
    num_embeddings=len(coco_train.target_transform.vocabulary),
    embedding_dim=8,
    encoder_dim=feature_mean.shape[-1],
    decoder_dim=16,
    attention_dim=4
)

In [6]:
embeddings = decoder.word_embedding(caption_batch)

In [7]:
print(f"One-hot encoded caption shape = {caption_batch[0].shape}")
print(f"Embedding shape = {embeddings[0].shape}")

One-hot encoded caption shape = torch.Size([51])
Embedding shape = torch.Size([51, 8])


In [8]:
print(caption_batch[0])
print(embeddings[0])

tensor([10000,    12,    78,    22,    34,   850,    31,     4,     0,   182,
            1,  1421,    65,     2,     0,     8,   283,  1486,     0,    34,
          204,     9,  1584,    47,    34,   924,     1,    34,  1118,     5,
          133,   167,    62,    34,  1186,     4,   121,     4,    36,     2,
          244,     6,    12,  1186,     1,   206,     6,   314, 10002,   139,
        10001])
tensor([[-0.0578, -0.3274,  0.0452,  0.2562, -0.6340,  0.6344,  1.5491, -1.5397],
        [ 0.6806, -0.2123,  0.6728,  1.1356, -1.5377,  0.6209, -0.0698, -0.1194],
        [-1.8721,  0.8615, -0.4453, -1.2441, -0.8107,  1.0677,  0.4041,  0.2136],
        [-1.4957,  0.4260,  0.4293,  0.6619,  0.3041, -0.1828, -0.3803,  1.1555],
        [ 0.5796,  0.0395, -0.2423,  0.2418, -1.2932, -0.6176,  0.9180,  0.0618],
        [-0.2173, -0.1386,  0.9177, -1.8113,  0.1931,  1.4906,  0.0167,  1.6835],
        [ 0.1837, -0.2586,  0.0316,  1.5410,  0.6644, -0.2807, -1.0454, -0.3785],
        [ 0.2360, -0

In [9]:
h = decoder.init_h(feature_mean)
c = decoder.init_c(feature_mean)

print(f"Initial h shape = {h.shape}")
print(f"Initial c shape = {c.shape}")

Initial h shape = torch.Size([2, 16])
Initial c shape = torch.Size([2, 16])


In [10]:
# Initial h, c of LSTM computed by MLP(feature_maps_mean)
print(h[0])
print(c[0])

tensor([-0.2004,  0.0623,  0.1886, -0.3241, -0.0010, -0.1566, -0.1023,  0.0371,
         0.1622,  0.1004,  0.1798,  0.1078,  0.1123, -0.0589,  0.0808,  0.0630],
       grad_fn=<SelectBackward>)
tensor([-0.0363,  0.0113, -0.3100,  0.0040,  0.1362,  0.0223, -0.3575, -0.1034,
        -0.2105,  0.0493,  0.1617,  0.0965, -0.0126, -0.2947, -0.0936,  0.0915],
       grad_fn=<SelectBackward>)


In [11]:
# How to get word embeddings of words at particular index of a caption in batch
index = 50
embeddings[:, index]

tensor([[-1.2501,  1.2775,  0.2789, -0.4453,  1.0922,  0.9080, -0.3000,  0.3264],
        [-1.2501,  1.2775,  0.2789, -0.4453,  1.0922,  0.9080, -0.3000,  0.3264]],
       grad_fn=<SelectBackward>)

In [12]:
feature_mean

tensor([[0.1412, 0.1195, 0.1757, 0.1956, 0.2014, 0.1619, 0.1296, 0.1212, 0.1625,
         0.2444, 0.2788, 0.3305, 0.3097, 0.3634, 0.2017, 0.1454, 0.0949, 0.0646,
         0.0738, 0.0580, 0.0685, 0.0834, 0.1497, 0.3198, 0.2564, 0.2706, 0.2274,
         0.2668, 0.4305, 0.2778, 0.3106, 0.2719, 0.2167, 0.0923, 0.0908, 0.1187,
         0.3236, 0.6264, 0.3948, 0.4090, 0.3418, 0.3622, 0.3230, 0.1912, 0.2707,
         0.2215, 0.1806, 0.0852, 0.0931, 0.1090, 0.2908, 0.4428, 0.1921, 0.2183,
         0.2172, 0.3397, 0.3572, 0.2309, 0.2297, 0.1774, 0.2011, 0.1361, 0.1507,
         0.1538, 0.2483, 0.2350, 0.1305, 0.1665, 0.2978, 0.4957, 0.3393, 0.1802,
         0.1848, 0.1314, 0.2549, 0.3474, 0.3141, 0.2434, 0.2089, 0.1747, 0.2000,
         0.2210, 0.3853, 0.5794, 0.2008, 0.1367, 0.1783, 0.1296, 0.2880, 0.4784,
         0.5109, 0.4420, 0.3456, 0.3144, 0.3219, 0.3269, 0.3614, 0.5044, 0.2522,
         0.1661, 0.2265, 0.1435, 0.2713, 0.4259, 0.4878, 0.4517, 0.3450, 0.2899,
         0.2741, 0.2950, 0.3

In [13]:
# How to concatenate context and embedding word
torch.cat([embeddings[:, index], feature_mean], dim=1)

tensor([[-1.2501,  1.2775,  0.2789, -0.4453,  1.0922,  0.9080, -0.3000,  0.3264,
          0.1412,  0.1195,  0.1757,  0.1956,  0.2014,  0.1619,  0.1296,  0.1212,
          0.1625,  0.2444,  0.2788,  0.3305,  0.3097,  0.3634,  0.2017,  0.1454,
          0.0949,  0.0646,  0.0738,  0.0580,  0.0685,  0.0834,  0.1497,  0.3198,
          0.2564,  0.2706,  0.2274,  0.2668,  0.4305,  0.2778,  0.3106,  0.2719,
          0.2167,  0.0923,  0.0908,  0.1187,  0.3236,  0.6264,  0.3948,  0.4090,
          0.3418,  0.3622,  0.3230,  0.1912,  0.2707,  0.2215,  0.1806,  0.0852,
          0.0931,  0.1090,  0.2908,  0.4428,  0.1921,  0.2183,  0.2172,  0.3397,
          0.3572,  0.2309,  0.2297,  0.1774,  0.2011,  0.1361,  0.1507,  0.1538,
          0.2483,  0.2350,  0.1305,  0.1665,  0.2978,  0.4957,  0.3393,  0.1802,
          0.1848,  0.1314,  0.2549,  0.3474,  0.3141,  0.2434,  0.2089,  0.1747,
          0.2000,  0.2210,  0.3853,  0.5794,  0.2008,  0.1367,  0.1783,  0.1296,
          0.2880,  0.4784,  

In [14]:
predictions, contexts = decoder.forward(feature_maps, feature_mean, caption_batch)

In [15]:
caption_batch.shape

torch.Size([2, 51])

Number of predicted words == caption_len - 1

In [16]:
print(predictions.shape)

torch.Size([50, 2, 10004])


Random guess prediction probability

In [21]:
predictions[0, 0, :100]

tensor([4.5802e-05, 5.6117e-05, 4.4048e-05, 6.3060e-05, 6.3240e-05, 2.4295e-05,
        9.3775e-05, 1.2415e-04, 7.2890e-05, 1.2004e-04, 7.5943e-05, 2.2056e-04,
        2.5207e-04, 9.8295e-05, 1.3943e-04, 7.3220e-05, 1.0729e-04, 7.2134e-05,
        1.6759e-04, 7.6803e-05, 9.3828e-05, 7.8082e-05, 7.1193e-05, 3.9398e-05,
        3.5730e-05, 1.0169e-04, 6.4523e-05, 7.8792e-05, 6.6139e-05, 7.7558e-05,
        1.4172e-04, 1.2873e-04, 8.9832e-05, 6.9432e-05, 1.7846e-04, 4.9560e-05,
        5.3852e-05, 6.7340e-05, 6.4957e-05, 1.0165e-04, 9.8409e-05, 6.0714e-05,
        1.1532e-04, 1.7585e-04, 1.0632e-04, 1.0307e-04, 1.0356e-04, 1.1976e-04,
        5.0463e-05, 1.2893e-04, 8.9859e-05, 3.1828e-05, 6.2140e-05, 1.2632e-04,
        5.0690e-05, 7.1590e-05, 8.8106e-05, 5.5537e-05, 3.2028e-05, 6.3830e-05,
        3.4250e-05, 5.5032e-05, 8.9259e-05, 7.1087e-05, 8.5833e-05, 4.3138e-05,
        1.0140e-04, 5.8886e-05, 1.1885e-04, 1.6113e-04, 1.0205e-04, 1.1345e-04,
        1.1555e-04, 1.0366e-04, 6.8066e-

In [20]:
1 / 10_004

9.996001599360256e-05

In [22]:
contexts.shape

torch.Size([50, 2, 196])

Contexts output

In [26]:
sample_context = contexts[0, 0].detach()
sample_context = sample_context.view(14, 14).numpy()
sample_context

array([[0.15760812, 0.12856914, 0.17816314, 0.1867098 , 0.18844461,
        0.1513915 , 0.12745804, 0.12468888, 0.16500652, 0.24868344,
        0.29475075, 0.37104458, 0.35524386, 0.41509762],
       [0.22528324, 0.16174345, 0.11304213, 0.06997125, 0.0699434 ,
        0.06107442, 0.07879632, 0.09875125, 0.18710577, 0.3577407 ,
        0.28041577, 0.31577718, 0.269769  , 0.30388695],
       [0.47945535, 0.3343305 , 0.37012273, 0.306884  , 0.23691249,
        0.11510529, 0.12544821, 0.16009569, 0.3920914 , 0.72889733,
        0.47876254, 0.4967263 , 0.4178265 , 0.39918992],
       [0.3665487 , 0.23079944, 0.32419032, 0.27719703, 0.22037895,
        0.10367082, 0.12389934, 0.1400319 , 0.3218997 , 0.49639863,
        0.22686902, 0.24014634, 0.23531097, 0.330303  ],
       [0.40279853, 0.30246395, 0.29192436, 0.20803887, 0.21314445,
        0.13724275, 0.1733399 , 0.17690325, 0.26021236, 0.25225908,
        0.1525959 , 0.16419768, 0.2701406 , 0.43512106],
       [0.39304125, 0.2573224 , 0.2

In [ ]:
plt.title(f"Sample context obtrain from forward pass", fontsize=20, y=1.025)
sns.heatmap(sample_context, annot=True, square=True, cmap=plt.rcParams["image.cmap"], vmin=min_value, vmax=max_value)